In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet

# API Connection
from sodapy import Socrata

client = Socrata("data.cityofnewyork.us", "yUnYRTjXHWdK6ATxbXOsJR52v", timeout=60)

# First 5000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("qgea-i56i", limit=20000)
df = pd.DataFrame.from_records(results)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   cmplnt_num                   5000 non-null   object
 1   cmplnt_fr_dt                 5000 non-null   object
 2   cmplnt_fr_tm                 5000 non-null   object
 3   cmplnt_to_tm                 5000 non-null   object
 4   addr_pct_cd                  5000 non-null   object
 5   rpt_dt                       5000 non-null   object
 6   ky_cd                        5000 non-null   object
 7   ofns_desc                    5000 non-null   object
 8   pd_cd                        4996 non-null   object
 9   pd_desc                      5000 non-null   object
 10  crm_atpt_cptd_cd             5000 non-null   object
 11  law_cat_cd                   5000 non-null   object
 12  boro_nm                      5000 non-null   object
 13  loc_of_occur_desc            5000

In [5]:
#view columns before dropping
df.columns

Index(['cmplnt_num', 'cmplnt_fr_dt', 'cmplnt_fr_tm', 'cmplnt_to_tm',
       'addr_pct_cd', 'rpt_dt', 'ky_cd', 'ofns_desc', 'pd_cd', 'pd_desc',
       'crm_atpt_cptd_cd', 'law_cat_cd', 'boro_nm', 'loc_of_occur_desc',
       'prem_typ_desc', 'juris_desc', 'jurisdiction_code', 'parks_nm',
       'hadevelopt', 'housing_psa', 'x_coord_cd', 'y_coord_cd',
       'susp_age_group', 'susp_race', 'susp_sex', 'latitude', 'longitude',
       'lat_lon', 'patrol_boro', 'station_name', 'vic_age_group', 'vic_race',
       'vic_sex', ':@computed_region_efsh_h5xi', ':@computed_region_f5dn_yrer',
       ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q',
       ':@computed_region_sbqj_enih', 'cmplnt_to_dt', 'transit_district'],
      dtype='object')

In [6]:
# Drop columns that are not needed
df.drop(columns=[":@computed_region_efsh_h5xi",
                  ":@computed_region_f5dn_yrer", 
                  ":@computed_region_yeji_bk3q", 
                  ":@computed_region_92fq_4b7q", 
                  ":@computed_region_sbqj_enih", 
                  "station_name",
                  "latitude",
                  "longitude",
                  "lat_lon",
                  'x_coord_cd',
                  'y_coord_cd',
                  'transit_district',
                  'pd_cd',
                  'ky_cd',
                  'cmplnt_num'])


,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_tm,addr_pct_cd,rpt_dt,ofns_desc,pd_desc,crm_atpt_cptd_cd,law_cat_cd,boro_nm,...,hadevelopt,housing_psa,susp_age_group,susp_race,susp_sex,patrol_boro,vic_age_group,vic_race,vic_sex,cmplnt_to_dt
0,2022-12-31T00:00:00.000,09:00:00,(null),13,2022-12-31T00:00:00.000,SEX CRIMES,"SEXUAL ABUSE 3,2",ATTEMPTED,MISDEMEANOR,MANHATTAN,...,(null),44346,UNKNOWN,UNKNOWN,U,PATROL BORO MAN SOUTH,<18,UNKNOWN,F,NaN
1,2022-12-31T00:00:00.000,02:00:00,02:15:00,44,2022-12-31T00:00:00.000,OFFENSES AGAINST PUBLIC ADMINI,VIOLATION OF ORDER OF PROTECTI,COMPLETED,MISDEMEANOR,BRONX,...,(null),(null),25-44,WHITE HISPANIC,M,PATROL BORO BRONX,25-44,WHITE HISPANIC,F,2022-12-31T00:00:00.000
2,2022-12-31T00:00:00.000,17:20:00,(null),14,2022-12-31T00:00:00.000,VEHICLE AND TRAFFIC LAWS,LEAVING SCENE-ACCIDENT-PERSONA,COMPLETED,MISDEMEANOR,MANHATTAN,...,(null),(null),(null),(null),(null),PATROL BORO MAN SOUTH,18-24,ASIAN / PACIFIC ISLANDER,F,NaN
3,2022-12-29T00:00:00.000,20:20:00,22:15:00,114,2022-12-31T00:00:00.000,CRIMINAL MISCHIEF & RELATED OF,RECKLESS ENDANGERMENT OF PROPE,COMPLETED,MISDEMEANOR,QUEENS,...,(null),1193,(null),(null),(null),PATROL BORO QUEENS NORTH,UNKNOWN,UNKNOWN,E,2022-12-29T00:00:00.000
4,2022-12-31T00:00:00.000,16:30:00,16:40:00,101,2022-12-31T00:00:00.000,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,QUEENS,...,(null),34703,25-44,BLACK,F,PATROL BORO QUEENS SOUTH,45-64,BLACK,F,2022-12-31T00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2022-12-28T00:00:00.000,20:00:00,20:25:00,32,2022-12-28T00:00:00.000,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,MANHATTAN,...,(null),(null),UNKNOWN,BLACK,F,PATROL BORO MAN NORTH,25-44,BLACK,F,2022-12-28T00:00:00.000
4996,2022-12-27T00:00:00.000,23:48:00,00:50:00,63,2022-12-28T00:00:00.000,BURGLARY,"BURGLARY,COMMERCIAL,NIGHT",COMPLETED,FELONY,BROOKLYN,...,(null),(null),UNKNOWN,UNKNOWN,M,PATROL BORO BKLYN SOUTH,UNKNOWN,UNKNOWN,D,2022-12-28T00:00:00.000
4997,2022-12-28T00:00:00.000,16:58:00,17:00:00,77,2022-12-28T00:00:00.000,GAMBLING,"GAMBLING, DEVICE, POSSESSION",COMPLETED,MISDEMEANOR,BROOKLYN,...,(null),(null),45-64,ASIAN / PACIFIC ISLANDER,M,PATROL BORO BKLYN NORTH,UNKNOWN,UNKNOWN,E,2022-12-28T00:00:00.000
4998,2022-12-27T00:00:00.000,20:40:00,(null),83,2022-12-28T00:00:00.000,OFFENSES AGAINST PUBLIC ADMINI,"CONTEMPT,CRIMINAL",COMPLETED,MISDEMEANOR,BROOKLYN,...,(null),(null),25-44,UNKNOWN,M,PATROL BORO BKLYN NORTH,25-44,WHITE,F,NaN
